In [1]:
from termcolor import cprint

In [2]:
# Test packets from WD:
tps = [
    b"\xd5\x04\x08\x18\x00\x03\x03\n\x00\x18\x00\x00\x00\x00\x00`\x00B\x80\x0f\xc1\x00\xb8\xf4\x014!\xfd\x0cV\x0e\x18\r\x10'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00",
    b"\xd5\x04\x08\x18\x00\x04\x03\n\x00\x18\x00\x00\x00\x00\x00`\x00B\x80\x0f\xc1\x00\xb8\xf4\x014!\xfd\x0cV\x0e\x18\r\x10'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00",
    b"\xd5\x04\x08\x18\x00\x05\x03\n\x00\x18\x00\x00\x00\x00\x00`\x00B\x80\x0f\xc1\x00\xb8\xf4\x014!\xfd\x0cV\x0e\x18\r\x10'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00",
    b"\xd5\x04\x08\x18\x00\x06\x03\n\x00\x18\x00\x00\x00\x00\x00\xe0\x00B\x80\x0f\xc1\x00\xb8\xf4\x014!\xfd\x0cV\x0e\x18\r\x10'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"
]
[len(tp) for tp in tps]

[47, 47, 47, 47]

In [3]:
def byte_to_binstr(B):
    # Convert to bin string:
    binstr = bin(B)[2:]
    # Pad with appropriate number of zeros:
    binstr = '0' * (8-len(binstr)) + binstr
    return binstr

def bytes_to_binstr(Bs):
    return '0b '+' '.join(byte_to_binstr(b) for b in Bs)

In [4]:
from scapy.utils import hexdump
for i,tp in enumerate(tps):
    print(f'\nTest Packet {i}: ')
    print(bytes_to_binstr(tp))
    hexdump(tp)


Test Packet 0: 
0b 11010101 00000100 00001000 00011000 00000000 00000011 00000011 00001010 00000000 00011000 00000000 00000000 00000000 00000000 00000000 01100000 00000000 01000010 10000000 00001111 11000001 00000000 10111000 11110100 00000001 00110100 00100001 11111101 00001100 01010110 00001110 00011000 00001101 00010000 00100111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
0000  D5 04 08 18 00 03 03 0A 00 18 00 00 00 00 00 60  ...............`
0010  00 42 80 0F C1 00 B8 F4 01 34 21 FD 0C 56 0E 18  .B.......4!..V..
0020  0D 10 27 00 00 00 00 00 00 00 00 00 00 00 00     ..'............

Test Packet 1: 
0b 11010101 00000100 00001000 00011000 00000000 00000100 00000011 00001010 00000000 00011000 00000000 00000000 00000000 00000000 00000000 01100000 00000000 01000010 10000000 00001111 11000001 00000000 10111000 11110100 00000001 00110100 00100001 11111101 00001100 01010110 00001110 00011000 00001101 00010000 00100111 0000000

In [18]:
from trans_tools import *
for i,tp in enumerate(tps):
    print(f'Test Packet {i}: ')
    packet = parse_packet(tp)
    print(packet)

Test Packet 0: 
#3 [RS_KEEP_ALIVE + NOT_DEPLOYED] 	CHARGER: OFF 	BATTERIES: TEMP_CONNECTED 	UART0: OFF 	UART1: INITIALIZED_&_ACTIVE
1V2: BAD 	1V8: BAD 	2V5: 0.0 	2V8: 0.0 	3V3: BAD 	5V0: BAD 	24V: 0.41
VBm: 3.32V 	VL: 23.82V 	Vcc28: 26.10V 	VSA: 1.98V 	ISA: 7.4mA
HEATER [268K |274K |273K] 	 is HEATING, Control: ENABLED 	@ 0.000W / 0.695W 	Period: 1.2ms
Tbatt: 428.1K ± 0.0K 	Tchrg: 428.1K ± 0.0K 	Tblimp: 0.0K
 D5 04 08 18 00 03 03 0A 00 18 00 00 00 00 00 60 00 42 80 0F C1 00 B8 F4 01 34 21 FD 0C 56 0E 18 0D 10 27 00 00 00 00 00 00 00 00 00 00 00 00  ...............`.B.......4!..V....'............
Test Packet 1: 
#4 [RS_KEEP_ALIVE + NOT_DEPLOYED] 	CHARGER: OFF 	BATTERIES: TEMP_CONNECTED 	UART0: OFF 	UART1: INITIALIZED_&_ACTIVE
1V2: BAD 	1V8: BAD 	2V5: 0.0 	2V8: 0.0 	3V3: BAD 	5V0: BAD 	24V: 0.41
VBm: 3.32V 	VL: 23.82V 	Vcc28: 26.10V 	VSA: 1.98V 	ISA: 7.4mA
HEATER [268K |274K |273K] 	 is HEATING, Control: ENABLED 	@ 0.000W / 0.695W 	Period: 1.2ms
Tbatt: 428.1K ± 0.0K 	Tchrg: 428.1K ± 0.0K

In [6]:
def create_adc_sample(val: int, n_top: int, n_bot: int, n_adc_bits: int = 12):
    # Make `val` at most `n_adc_bits` bits long:
    val = val & ((1<<n_adc_bits)-1)
    # Keep only the top `n_top` bits:
    val = val >> (n_adc_bits-n_top)
    # Keep only the bottom `n_bot` bits:
    val = val & ((1<<n_bot)-1)
    return val

# test: `create_adc_sample`
def test_top_selection():
    # Make sure it can accurately select the indicated number of top bits and 
    # only the top bits for a wide range of number bitlengths:
    for n_bits in range(1,65):
        for n_top in range(1,n_bits):
            # Create a test value that's `n_bits` long with 1s only in the top `n_top` bits:
            x = ((1<<n_top)-1) << (n_bits-n_top)
            val = create_adc_sample(x, n_top, n_bot = n_bits, n_adc_bits = n_bits)
            # Make sure the right number of bits were selected:
            assert len(bin(val))-2 == n_top, f"{x=}={bin(x)}, {n_top=}, {n_bits=}, {val=}={bin(val)}"
            # Make sure they actually came from the top (are all 1s):
            assert sum(int(b) for b in bin(x)[2:]) == n_top, f"{x=}, {n_top=}, {n_bits=}, {val=}={bin(val)}"
            
test_top_selection()
cprint('PASSED!', 'green')

def test_bottom_selection():
    # Make sure it can accurately select the indicated number of bottom bits and 
    # only the bottom bits for a wide range of number bitlengths:
    for n_bits in range(1,65):
        for n_bot in range(1,n_bits):
            # Create a test value with all 1s for the bottom `n_bot` bits:
            x = (1<<n_bot) - 1
            val = create_adc_sample(x, n_top = n_bits, n_bot = n_bot, n_adc_bits = n_bits)
            # Make sure the right number of bits were selected:
            assert len(bin(val))-2 == n_bot, f"{x=}={bin(x)}, {n_bot=}, {n_bits=}, {val=}={bin(val)}"
            # Make sure they actually came from the top (are all 1s):
            assert sum(int(b) for b in bin(x)[2:]) == n_bot, f"{x=}, {n_bot=}, {n_bits=}, {val=}={bin(val)}"
            
test_bottom_selection()
cprint('PASSED!', 'green')

PASSED!
PASSED!


In [7]:
# Make a reference test packet (where true input values are known):


In [8]:
from trans_tools import *

In [9]:
N_PACKET_BYTES: int = 46 # total # of bytes in detailed status packet
test_val = 1
packet_bytes = b'\xD5' + (test_val<<(8-1)).to_bytes(1, 'little') + b'\x00' * (N_PACKET_BYTES-1)
packet_bytes

b'\xd5\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [10]:
# Decode the packet:
packet = parse_packet(packet_bytes)
print(packet)

#0 [RS_INIT + NOT_DEPLOYED] 	CHARGER: OFF 	BATTERIES: DISCONNECTED 	UART0: OFF 	UART1: OFF
1V2: BAD 	1V8: BAD 	2V5: 0.0 	2V8: 0.0 	3V3: BAD 	5V0: GOOD 	24V: 0.00
VBm: 0.00V 	VL: 0.00V 	Vcc28: 0.00V 	VSA: 0.00V 	ISA: 0.0mA
HEATER [428K |428K |428K] 	 is NOT_HEATING, Control: DISABLED 	@ nanW / nanW 	Period: 0.0ms
Tbatt: 428.1K ± 0.0K 	Tchrg: 428.1K ± 0.0K 	Tblimp: 0.0K
 D5 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ...............................................


In [11]:
from collections import OrderedDict
# Fields to test:
# Each field has the following entries: (n_bits, is_adc_val, n_upper_adc_bits, n_lower_adc_bits)
FIELDS_TO_TEST: OrderedDict = OrderedDict([ # Added a couple extra fields (a-e) for sanity checking the packing
        ('a', (4, False, 0,0)),
        ('b', (4, False, 0,0)),
        ('c', (4, False, 0,0)),
        ('d', (3, False, 0,0)),
        ('e', (9, False, 0,0)),
        ('Io_ChargingStatus1', (1, False, 0,0)),
        ('Io_ChargingStatus2', (1, False, 0,0)),
        ('Io_BatteryConnectionStatus', (1, False, 0,0)),
        ('Io_BatteryLatchStatus', (1, False, 0,0)),
        ('Io_1V2PowerGood', (1, False, 0,0)),
        ('Io_1V8PowerGood', (1, False, 0,0)),
        ('Io_3V3PowerGood', (1, False, 0,0)),
        ('Io_5V0PowerGood', (1, False, 0,0)),
        ('Watchdog_State', (8, False, 0,0)),
        ('Watchdog_DeploymentStatus', (2, False, 0,0)),
        ('Watchdog_Uart0State', (1, False, 0,0)),
        ('Watchdog_Uart1State', (1, False, 0,0)),
        ('Adc_BatteryTempRaw', (12, True, 12,12)),
        ('Watchdog_DetailedHeartbeatSequenceNumber', (8, False, 0,0)),
        ('Watchdog_DigitalOutputStates', (32, False, 0,0)),
        ('Watchdog_ResetLogs', (40, False, 0,0)),
        ('Adc_LanderVoltageRaw', (7, True, 7,12)),
        ('Adc_BatteryChargingTempRaw', (9, True, 9,12)),
        ('Adc_FullSystemVoltageRaw', (5, True, 5,12)),
        ('Adc_FullSystemCurrentRaw', (9, True, 12,9)),
        ('Adc_SwitchedBatteryVoltageRaw', (9, True, 9,12)),
        ('Adc_Vcc24VoltageRaw', (7, True, 7,12)),
        ('Heater_ControlEnabled', (1, False, 0,0)),
        ('Heater_IsHeating', (1, False, 0,0)),
        ('Adc_2V5VoltageRaw', (5, True, 5,12)),
        ('Adc_2V8VoltageRaw', (5, True, 5,12)),
        ('Adc_Vcc28VoltageRaw', (6, True, 6,12)),
        ('Heater_Kp', (16, False, 0,0)),
        ('Heater_PwmLimit_DutyCycleCounter', (16, False, 0,0)),
        ('Heater_SetpointValue', (16, False, 0,0)),
        ('Heater_OnValue', (16, False, 0,0)),
        ('Heater_OffValue', (16, False, 0,0)),
        ('Heater_DutyCyclePeriodCycles', (16, False, 0,0)),
        ('Heater_DutyCycleCounter', (16, False, 0,0)),
        ('I2C_BatteryChargeRaw', (16, False, 0,0)),
        ('I2C_BatteryVoltageRaw', (16, False, 0,0)),
        ('I2C_BatteryCurrentRaw', (16, False, 0,0)),
        ('I2C_FuelGaugeTempRaw', (16, False, 0,0)),
        ('I2C_BatteryChargeTelemRaw', (8, False, 0,0)),
        ('I2C_BatteryCurrentTelemRaw', (8, False, 0,0))
    ])

In [12]:
N_PACKET_BYTES: int = math.ceil(sum(n for n, *_ in FIELDS_TO_TEST.values())/8) # total # of bytes in detailed status packet
def create_bitfield(test_vals: Dict[str,int]):
    x: int = 0
    idx_bit_head: int = 0
    for field_name, (n_bits, is_adc_val, n_upper_adc_bits, n_lower_adc_bits) in FIELDS_TO_TEST.items():
        field_val = test_vals[field_name]
        if is_adc_val:
            field_val = create_adc_sample(field_val, n_upper_adc_bits, n_lower_adc_bits)
        x |= field_val << idx_bit_head
        idx_bit_head += n_bits
    return x.to_bytes(N_PACKET_BYTES, 'little')

# Test `create_bitfield` against a couple packets which were compiled on an MSP430:
# Create a dict with all fields from FIELDS_TO_TEST and a default value of 0:
blank_vals = dict((f,0) for f in FIELDS_TO_TEST.keys())
# Create pairs of test tuples of input values into MSP430 struct and output byte streams:
test_pairs: List[Tuple[ Dict[str,int], bytes ]] = [
    ({**blank_vals, 'a':1,'b':2,'c':3, 'd':4, 'e':0b111100001, 'Io_ChargingStatus2':1, 'Adc_BatteryTempRaw':0b100000001111},b'!\xc3\xf0\x02\x00\xf0\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
    ({**blank_vals, 'a':1,'b':2,'c':3, 'd':4, 'e':0b111100001, 'Io_ChargingStatus2':1, 'Adc_BatteryTempRaw':0b100000001111, 'Adc_LanderVoltageRaw':0b111111101101},b'!\xc3\xf0\x02\x00\xf0\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
    ({**blank_vals, 'a':1,'b':2,'c':3, 'd':4, 'e':0b111100001, 'Io_ChargingStatus2':1, 'Adc_BatteryTempRaw':0b100000001111, 'Adc_LanderVoltageRaw': 0b111111101101, 'Adc_BatteryChargingTempRaw': 0b111110101011, 'Adc_FullSystemVoltageRaw': 0b111110101011, 'Adc_FullSystemCurrentRaw': 0b111110101011, 'Adc_SwitchedBatteryVoltageRaw': 0b111110101011, 'Adc_Vcc24VoltageRaw': 0b111110101011, 'Adc_2V5VoltageRaw': 0b111110101011, 'Adc_2V8VoltageRaw': 0b111110101011, 'Adc_Vcc28VoltageRaw': 0b111110101011},b'!\xc3\xf0\x02\x00\xf0\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xfa\x7fu\xfd>\xff\xfb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
    ({**blank_vals, 'a':1,'b':2,'c':3, 'd':4, 'e':0b111100001, 'Io_ChargingStatus2':1, 'Adc_BatteryTempRaw':0b100000001111, 'Adc_LanderVoltageRaw': 0b111111101101, 'Adc_BatteryChargingTempRaw': 0b111110101011, 'Adc_FullSystemVoltageRaw': 0b111110101011, 'Adc_FullSystemCurrentRaw': 0b111110101011, 'Adc_SwitchedBatteryVoltageRaw': 0b111110101011, 'Adc_Vcc24VoltageRaw': 0b111110101011, 'Adc_2V5VoltageRaw': 0b111110101011, 'Adc_2V8VoltageRaw': 0b111110101011, 'Adc_Vcc28VoltageRaw': 0b111110101011,
        'Watchdog_State': 5,
        'Watchdog_DeploymentStatus': 1,
        'Watchdog_DetailedHeartbeatSequenceNumber': 15,
        'Watchdog_DigitalOutputStates': 0b00001111101010100011001111111111,
        'Heater_Kp': 0x0A0B,
        'Heater_PwmLimit_DutyCycleCounter': 0x0C0D,
        'Heater_SetpointValue': 0x0E0F,
        'Heater_OnValue': 0xAABB,
        'Heater_OffValue': 0xCCDD,
        'Heater_DutyCyclePeriodCycles': 0xEEFF,
        'Heater_DutyCycleCounter': 0xF001,
        'I2C_BatteryCurrentTelemRaw': 0x5B},
        b'!\xc3\xf0\x02\x05\xf1\x80\x0f\xff3\xaa\x0f\x00\x00\x00\x00\x00\xff\xfa\x7fu\xfd>\xff\xfb\x0b\n\r\x0c\x0f\x0e\xbb\xaa\xdd\xcc\xff\xee\x01\xf0\x00\x00\x00\x00\x00\x00\x00\x00\x00['
     )
]

# Print the last test packet for visual inspection:
print(bytes_to_binstr(test_pairs[-1][1]))
print(bytes_to_binstr(create_bitfield(test_pairs[-1][0])))

for tp in test_pairs:
    assert tp[1] == create_bitfield(tp[0])
cprint('PASSED!', 'green')



0b 00100001 11000011 11110000 00000010 00000101 11110001 10000000 00001111 11111111 00110011 10101010 00001111 00000000 00000000 00000000 00000000 00000000 11111111 11111010 01111111 01110101 11111101 00111110 11111111 11111011 00001011 00001010 00001101 00001100 00001111 00001110 10111011 10101010 11011101 11001100 11111111 11101110 00000001 11110000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011011
0b 00100001 11000011 11110000 00000010 00000101 11110001 10000000 00001111 11111111 00110011 10101010 00001111 00000000 00000000 00000000 00000000 00000000 11111111 11111010 01111111 01110101 11111101 00111110 11111111 11111011 00001011 00001010 00001101 00001100 00001111 00001110 10111011 10101010 11011101 11001100 11111111 11101110 00000001 11110000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 01011011
PASSED!


In [13]:
from collections import OrderedDict
# Fields to test:
# Each field has the following entries: (n_bits, is_adc_val, n_upper_adc_bits, n_lower_adc_bits)
FIELDS_TO_TEST: OrderedDict = OrderedDict([
        ('Io_ChargingStatus1', (1, False, 0,0)),
        ('Io_ChargingStatus2', (1, False, 0,0)),
        ('Io_BatteryConnectionStatus', (1, False, 0,0)),
        ('Io_BatteryLatchStatus', (1, False, 0,0)),
        ('Io_1V2PowerGood', (1, False, 0,0)),
        ('Io_1V8PowerGood', (1, False, 0,0)),
        ('Io_3V3PowerGood', (1, False, 0,0)),
        ('Io_5V0PowerGood', (1, False, 0,0)),
        ('Watchdog_State', (8, False, 0,0)),
        ('Watchdog_DeploymentStatus', (2, False, 0,0)),
        ('Watchdog_Uart0State', (1, False, 0,0)),
        ('Watchdog_Uart1State', (1, False, 0,0)),
        ('Adc_BatteryTempRaw', (12, True, 12,12)),
        ('Watchdog_DetailedHeartbeatSequenceNumber', (8, False, 0,0)),
        ('Watchdog_DigitalOutputStates', (32, False, 0,0)),
        ('Watchdog_ResetLogs', (40, False, 0,0)),
        ('Adc_LanderVoltageRaw', (7, True, 7,12)),
        ('Adc_BatteryChargingTempRaw', (9, True, 9,12)),
        ('Adc_FullSystemVoltageRaw', (5, True, 5,12)),
        ('Adc_FullSystemCurrentRaw', (9, True, 12,9)),
        ('Adc_SwitchedBatteryVoltageRaw', (9, True, 9,12)),
        ('Adc_Vcc24VoltageRaw', (7, True, 7,12)),
        ('Heater_ControlEnabled', (1, False, 0,0)),
        ('Heater_IsHeating', (1, False, 0,0)),
        ('Adc_2V5VoltageRaw', (5, True, 5,12)),
        ('Adc_2V8VoltageRaw', (5, True, 5,12)),
        ('Adc_Vcc28VoltageRaw', (6, True, 6,12)),
        ('Heater_Kp', (16, False, 0,0)),
        ('Heater_PwmLimit_DutyCycleCounter', (16, False, 0,0)),
        ('Heater_SetpointValue', (16, False, 0,0)),
        ('Heater_OnValue', (16, False, 0,0)),
        ('Heater_OffValue', (16, False, 0,0)),
        ('Heater_DutyCyclePeriodCycles', (16, False, 0,0)),
        ('Heater_DutyCycleCounter', (16, False, 0,0)),
        ('I2C_BatteryChargeRaw', (16, False, 0,0)),
        ('I2C_BatteryVoltageRaw', (16, False, 0,0)),
        ('I2C_BatteryCurrentRaw', (16, False, 0,0)),
        ('I2C_FuelGaugeTempRaw', (16, False, 0,0)),
        ('I2C_BatteryChargeTelemRaw', (8, False, 0,0)),
        ('I2C_BatteryCurrentTelemRaw', (8, False, 0,0))
    ])

In [14]:
## Make sure all possible values for each field are extracted correctly:

import random
N_PACKET_BYTES: int = math.ceil(sum(n for n, *_ in FIELDS_TO_TEST.values())/8) # total # of bytes in detailed status packet

# Create a dict with all fields from FIELDS_TO_TEST and a default value of 0:
blank_vals = dict((f,0) for f in FIELDS_TO_TEST.keys())

print_test_packet_bits: bool = False
idx_bit_head = 0 # index of bit in the test packet at which to start inserting the value
for test_field_name, (test_field_bits, is_adc_val, n_upper_adc_bits, n_lower_adc_bits) in [*FIELDS_TO_TEST.items()]:
    idx_byte_head = idx_bit_head // 8 # index of the byte this starts after
    idx_bit_head_in_byte = idx_bit_head % 8 # how many bits into the byte this starts
    total_val_bytes_spanned = math.ceil((idx_bit_head_in_byte + test_field_bits)/8) # total number of bytes this value will span
    print(f"Testing field {test_field_name} with {test_field_bits}b at {idx_bit_head=} -> {idx_byte_head}B+{idx_bit_head_in_byte}b")
    
    ## Make a test packet for all possible values for the field:
    if is_adc_val:
        test_vals = range(0, 1<<12) # go up to ADC max num of bits even if field truncates
    elif test_field_bits > 12:
        # Sample only 2^12 points from the entire valid range:
        test_vals = random.sample(range(0, 1<<test_field_bits), 1<<12)
    else:
        test_vals = range(0, 1<<test_field_bits)
    for test_val in test_vals:
        # Start with a packet of all zeros:
        # Create a dict with all fields 0 except `test_field_name`, which has the specified test value:
        test_vals = {**blank_vals, test_field_name: test_val}
        # Create bitfield packet and add the `DetailedStatus` flag at the start:
        packet_bytes = b'\xD5' + create_bitfield(test_vals)
        
        # Print the bits of the compiled packet:
        if print_test_packet_bits:
            print(bytes_to_binstr(packet_bytes))
        
        ## Decode the packet:
        packet = parse_packet(packet_bytes)
        
        # Ensure only the designated field can have a non-zero value and that its value is correct:
        for checking_field_name in packet.BITFIELD_ALLOCATIONS.keys():
            checking_field_val = getattr(packet.custom_payload, checking_field_name)
            
            if checking_field_name == test_field_name:
                if is_adc_val:
                    # Cut off any bits as necessary for transmission:
                    expected_val = create_adc_sample(test_val, n_upper_adc_bits, n_lower_adc_bits)
                    # Shift back out to full scale:
                    expected_val = expected_val << (12 - n_upper_adc_bits)
                elif test_field_name in ['I2C_BatteryChargeRaw', 'I2C_BatteryVoltageRaw', 'I2C_BatteryCurrentRaw', 'I2C_FuelGaugeTempRaw']:
                    # Flip endianness since these values are actually uint8_t[2] not uint16_t
                    expected_val = test_val
                    expected_val = int.from_bytes(expected_val.to_bytes(2, 'little'), 'big')
                else:
                    expected_val = test_val
            else:
                expected_val = 0
            
            err_msg = f"testing {checking_field_name=}, {checking_field_val=} <-> {test_field_name=}, {test_val=} => {expected_val=}"
            assert checking_field_val == expected_val, err_msg
    
    # Advance the bit head:
    idx_bit_head += test_field_bits
    
cprint('PASSED!', 'green')


Testing field Io_ChargingStatus1 with 1b at idx_bit_head=0 -> 0B+0b
Testing field Io_ChargingStatus2 with 1b at idx_bit_head=1 -> 0B+1b
Testing field Io_BatteryConnectionStatus with 1b at idx_bit_head=2 -> 0B+2b
Testing field Io_BatteryLatchStatus with 1b at idx_bit_head=3 -> 0B+3b
Testing field Io_1V2PowerGood with 1b at idx_bit_head=4 -> 0B+4b
Testing field Io_1V8PowerGood with 1b at idx_bit_head=5 -> 0B+5b
Testing field Io_3V3PowerGood with 1b at idx_bit_head=6 -> 0B+6b
Testing field Io_5V0PowerGood with 1b at idx_bit_head=7 -> 0B+7b
Testing field Watchdog_State with 8b at idx_bit_head=8 -> 1B+0b
Testing field Watchdog_DeploymentStatus with 2b at idx_bit_head=16 -> 2B+0b
Testing field Watchdog_Uart0State with 1b at idx_bit_head=18 -> 2B+2b
Testing field Watchdog_Uart1State with 1b at idx_bit_head=19 -> 2B+3b
Testing field Adc_BatteryTempRaw with 12b at idx_bit_head=20 -> 2B+4b
Testing field Watchdog_DetailedHeartbeatSequenceNumber with 8b at idx_bit_head=32 -> 4B+0b
Testing field Wa

In [15]:
# {d: getattr(packet.custom_payload, d) for d in dir(packet.custom_payload)}

In [16]:
N_PACKET_BYTES: int = 46 # total # of bytes in detailed status packet

# Start with a packet of all zeros:
packet_bytes = bytearray(b'\x00' * N_PACKET_BYTES)

idx_bit_head = 5
idx_byte_head = idx_bit_head // 8

test_val = 1
insert_val = test_val << (7 - idx_bit_head % 8)

packet_bytes[idx_byte_head] = insert_val

# Add the `DetailedStatus` flag at the beginning:
packet_bytes = b'\xD5' + packet_bytes

print(bytes_to_binstr(packet_bytes))
hexdump(packet_bytes)

0b 11010101 00000100 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
0000  D5 04 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0010  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0020  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00     ...............


In [17]:
n_bits = 12
n_bytes = math.ceil(n_bits / 8)
n_bits_in_final_byte = n_bits % 8

test_val = 1<<(n_bits-1)

bs = test_val.to_bytes(n_bytes, 'little')
bits_list = [byte_to_binstr(b) for b in bs]
bits_list[-1] = bits_list[-1][-n_bits_in_final_byte:]
Bs = int(''.join(bits_list),2).to_bytes(n_bytes, byteorder='little')
bytes_to_binstr(Bs)


'0b 00001000 00000000'